
# Task 7 – Basic Sales Summary from SQLite using Python

**Objective:**  
Use SQL inside Python to pull simple sales info (total quantity & total revenue per product) and display it via prints + a simple bar chart.

---


In [ ]:

import sqlite3
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:

# Connect to (or create) the SQLite database
conn = sqlite3.connect("sales_data.db")
cur = conn.cursor()

# Create the sales table if it does not exist
cur.execute("""
CREATE TABLE IF NOT EXISTS sales (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    product TEXT NOT NULL,
    quantity INTEGER NOT NULL,
    price REAL NOT NULL,
    sale_date TEXT NOT NULL
);
""")
conn.commit()
print("Database connected and table ensured.")


In [ ]:

# Insert sample rows only if table is empty
cur.execute("SELECT COUNT(*) FROM sales;")
if cur.fetchone()[0] == 0:
    sample_rows = [
        ("Shampoo", 10, 120.0, "2025-08-01"),
        ("Shampoo", 7, 120.0, "2025-08-02"),
        ("Conditioner", 5, 150.0, "2025-08-02"),
        ("Conditioner", 9, 150.0, "2025-08-03"),
        ("FaceWash", 12, 90.0, "2025-08-01"),
        ("FaceWash", 4, 90.0, "2025-08-03"),
        ("Serum", 6, 300.0, "2025-08-02"),
        ("Serum", 3, 300.0, "2025-08-03"),
        ("HairOil", 8, 200.0, "2025-08-01"),
        ("HairOil", 5, 200.0, "2025-08-03")
    ]
    cur.executemany(
        "INSERT INTO sales (product, quantity, price, sale_date) VALUES (?, ?, ?, ?);",
        sample_rows
    )
    conn.commit()
    print("Sample data inserted.")
else:
    print("Table already has data. Skipping insert.")


In [ ]:

# SQL to get total quantity and revenue per product
query = """
SELECT 
    product,
    SUM(quantity) AS total_qty,
    ROUND(SUM(quantity * price), 2) AS revenue
FROM sales
GROUP BY product
ORDER BY revenue DESC;
"""

# Load query results into Pandas DataFrame
df = pd.read_sql_query(query, conn)
df


In [ ]:

# Save the summary as CSV
df.to_csv("sales_summary_by_product.csv", index=False)
print("Summary saved to sales_summary_by_product.csv")


In [ ]:

plt.figure()
plt.bar(df["product"], df["total_qty"])
plt.title("Quantity by Product")
plt.xlabel("Product")
plt.ylabel("Total Quantity")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig("quantity_by_product.png")
plt.show()


In [ ]:

plt.figure()
plt.bar(df["product"], df["revenue"])
plt.title("Revenue by Product")
plt.xlabel("Product")
plt.ylabel("Revenue")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig("revenue_by_product.png")
plt.show()



## Interview Q&A

**How did you connect Python to a database?**  
Using `sqlite3.connect("sales_data.db")` to open a connection.

**What SQL query did you run? What does GROUP BY do?**  
We aggregated per `product` using `SUM(quantity)` and `SUM(quantity*price)`.  
`GROUP BY product` groups rows having the same product so aggregates work per product.

**How did you calculate revenue?**  
`SUM(quantity * price)` directly in SQL.

**How did you visualize the result?**  
Loaded the SQL result into a Pandas DataFrame and used Matplotlib `bar` charts.

**What does pandas do in your code?**  
It reads SQL results into a DataFrame (`pd.read_sql_query`) which is easy to print, save to CSV, and plot.

**Benefit of using SQL inside Python?**  
We can keep data in SQLite, run fast aggregations in SQL, and still use Python’s plotting and file I/O to automate analysis and reporting.
